# Forecasting Data

This is the **last** notebook of the series.

> Make sure you have executed all the previous ones. This notebook is entirely dependent upon the existance of a `model.pkl` file.
---

# Objective

At last, once trained, our model should be able to perform some predictions. This notebook should forecast some cases and perform analysis on the results obtained.

---